In [ ]:
# ==================== 라이브러리 임포트 섹션 ====================
# 운영체제 관련 기능을 사용하기 위한 라이브러리
import os
# 수치 연산을 위한 NumPy 라이브러리
import numpy as np
# PyTorch 딥러닝 프레임워크의 메인 모듈
import torch
# PyTorch의 신경망 구성 요소들 (레이어, 손실함수 등)
import torch.nn as nn
# PyTorch의 최적화 알고리즘들 (SGD, Adam 등)
import torch.optim as optim
# PyTorch의 컴퓨터 비전 관련 유틸리티
import torchvision
# 이미지 전처리 및 변환을 위한 모듈
import torchvision.transforms as transforms
# 데이터를 배치 단위로 로드하기 위한 DataLoader
from torch.utils.data import DataLoader
# 그래프 및 이미지 시각화를 위한 matplotlib
import matplotlib.pyplot as plt
# 이미지 처리를 위한 PIL (Python Imaging Library)
from PIL import Image
# 시간 측정을 위한 time 모듈
import time
# 랜덤 시드 설정을 위한 random 모듈
import random
# 파일 경로 패턴 매칭을 위한 glob 모듈
import glob
# 구글 코랩에서 구글 드라이브 마운트를 위한 모듈
from google.colab import drive
# 이미지 폴더 구조 기반 데이터셋 로딩을 위한 ImageFolder
from torchvision.datasets import ImageFolder
# 코랩에서 파일 다운로드를 위한 files 모듈
from google.colab import files
# 주피터/코랩 환경에서 HTML 표시를 위한 display 모듈
from IPython.display import display, HTML

# ==================== 구글 드라이브 마운트 섹션 ====================
# 구글 드라이브를 '/content/drive' 경로에 마운트하는 시도
try:
    # 구글 드라이브 마운트 실행
    drive.mount('/content/drive')
    # 마운트 성공 시 메시지 출력
    print("구글 드라이브 마운트 성공!")
# 마운트 실패 시 예외 처리
except:
    # 마운트 실패 또는 사용자가 취소한 경우 메시지 출력
    print("구글 드라이브 마운트 실패 또는 마운트 취소")

# ==================== 랜덤 시드 설정 섹션 ====================
# 재현 가능한 결과를 얻기 위해 모든 랜덤 시드를 고정하는 함수
def set_seed(seed):
    # Python 기본 random 모듈의 시드 설정
    random.seed(seed)
    # NumPy의 랜덤 시드 설정
    np.random.seed(seed)
    # PyTorch CPU 연산의 랜덤 시드 설정
    torch.manual_seed(seed)
    # CUDA(GPU)가 사용 가능한지 확인
    if torch.cuda.is_available():
        # 모든 GPU 장치의 랜덤 시드 설정
        torch.cuda.manual_seed_all(seed)
        # CUDA 연산을 결정론적으로 만들어 완전한 재현성 보장
        torch.backends.cudnn.deterministic = True

# 시드 값을 42로 설정하여 함수 호출
set_seed(42)

# ==================== 장치 설정 섹션 ====================
# CUDA가 사용 가능하면 GPU, 아니면 CPU를 학습 장치로 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 사용할 장치 정보 출력
print(f"사용 장치: {device}")

# ==================== 하이퍼파라미터 설정 섹션 ====================
# 한 번의 학습 반복(iteration)에서 처리할 이미지의 개수
batch_size = 64
# 생성할 이미지의 크기 (64x64 픽셀)
image_size = 64
# 생성자 네트워크의 입력 노이즈 벡터의 차원 수
nz = 100
# 생성자 네트워크의 특성 맵(feature map) 크기 조절 파라미터
ngf = 64
# 판별자 네트워크의 특성 맵 크기 조절 파라미터
ndf = 64
# 전체 데이터셋을 몇 번 반복해서 학습할지 결정하는 에폭 수
num_epochs = 50
# 경사하강법의 학습률 (너무 크면 불안정, 너무 작으면 학습 느림)
lr = 0.0003
# Adam 옵티마이저의 첫 번째 모멘텀 계수 (일반적으로 0.9이지만 GAN에서는 0.5 사용)
beta1 = 0.5

# ==================== 강아지 데이터셋 다운로드 함수 ====================
# CIFAR-10 데이터셋에서 강아지 이미지만 추출하여 저장하는 함수
def download_dog_dataset():
    # 사용자에게 다운로드 시작을 알리는 메시지 출력
    print("강아지 데이터셋 다운로드 중...")

    # torchvision의 datasets 모듈을 지역적으로 임포트
    import torchvision.datasets as datasets

    # CIFAR-10 데이터셋을 './cifar10' 폴더에 다운로드 (train=True: 훈련 데이터)
    cifar10 = datasets.CIFAR10(root='./cifar10', download=True, train=True)

    # CIFAR-10의 10개 클래스 라벨 목록을 가져옴
    class_labels = cifar10.classes
    # 클래스 목록을 출력하여 사용자에게 확인시켜줌
    print(f"CIFAR-10 클래스 목록: {class_labels}")

    # 'dog' 클래스의 인덱스를 찾음 (CIFAR-10에서 강아지는 인덱스 5)
    dog_idx = class_labels.index('dog')
    # 찾은 강아지 클래스 인덱스를 출력
    print(f"강아지 클래스 인덱스: {dog_idx}")

    # 강아지 이미지들을 저장할 빈 리스트 초기화
    dog_images = []

    # CIFAR-10 데이터셋의 모든 이미지를 순회
    for i in range(len(cifar10)):
        # i번째 이미지와 라벨을 가져옴
        img, label = cifar10[i]
        # 라벨이 강아지 인덱스와 같은지 확인
        if label == dog_idx:
            # 강아지 이미지면 리스트에 추가
            dog_images.append(img)

    # 추출된 강아지 이미지 개수를 출력
    print(f"{len(dog_images)}개의 강아지 이미지를 추출했습니다.")

    # 강아지 이미지를 저장할 폴더 생성 (exist_ok=True: 이미 존재해도 오류 없음)
    os.makedirs('./dog_dataset/dogs', exist_ok=True)

    # 추출된 모든 강아지 이미지를 파일로 저장
    for i, img in enumerate(dog_images):
        # i번째 강아지 이미지를 JPEG 형식으로 저장
        img.save(f'./dog_dataset/dogs/dog_{i}.jpg')

    # 저장 완료 메시지 출력
    print(f"이미지를 './dog_dataset/dogs/' 폴더에 저장했습니다.")
    # 저장된 데이터셋의 루트 경로 반환
    return './dog_dataset'

# ==================== 데이터셋 확인 및 로드 섹션 ====================
# 데이터셋 존재 여부를 확인하고 필요시 다운로드하는 시도
try:
    # './dog_dataset' 폴더가 존재하지 않거나 jpg 파일이 없는지 확인
    if not os.path.exists('./dog_dataset') or len(glob.glob('./dog_dataset/*/*.jpg')) == 0:
        # 조건에 해당하면 데이터셋 다운로드 함수 호출
        data_root = download_dog_dataset()
    # 데이터셋이 이미 존재하는 경우
    else:
        # 기존 데이터셋 경로를 사용
        data_root = './dog_dataset'
        # 기존 데이터셋 사용 메시지 출력
        print(f"기존 데이터셋 사용: {data_root}")
# 데이터셋 확인 중 오류가 발생한 경우
except:
    # 오류 발생 알림 메시지 출력
    print("데이터셋 확인 중 오류 발생, 다운로드를 시도합니다.")
    # 강제로 데이터셋 다운로드 함수 호출
    data_root = download_dog_dataset()

# ==================== 이미지 전처리 변환 정의 섹션 ====================
# 이미지 전처리 파이프라인을 정의 (여러 변환을 순차적으로 적용)
transform = transforms.Compose([
    # 이미지 크기를 image_size x image_size로 조정 (비율 유지하며 리사이즈)
    transforms.Resize(image_size),
    # 이미지 중앙 부분을 image_size x image_size로 잘라내기
    transforms.CenterCrop(image_size),
    # PIL 이미지를 PyTorch 텐서로 변환 (값 범위: [0, 1])
    transforms.ToTensor(),
    # 픽셀 값을 [-1, 1] 범위로 정규화 (평균=0.5, 표준편차=0.5)
    # 공식: output = (input - mean) / std = (input - 0.5) / 0.5
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# ==================== 데이터셋 로드 및 DataLoader 생성 섹션 ====================
# 이미지 폴더 기반 데이터셋 로드 시도
try:
    # ImageFolder를 사용하여 폴더 구조 기반으로 데이터셋 생성
    dataset = ImageFolder(root=data_root, transform=transform)
    # DataLoader 생성: 배치 단위로 데이터 로드, 셔플링, 멀티프로세싱
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    # 데이터셋 로드 완료 메시지와 총 이미지 수 출력
    print(f"데이터셋 로드 완료: {len(dataset)} 이미지")
# 데이터셋 로드 중 예외 발생시 처리
except Exception as e:
    # 발생한 오류 내용 출력
    print(f"데이터셋 로드 오류: {e}")
    # 임시 방편으로 랜덤 데이터 사용 안내
    print("임의 데이터로 코드를 계속 실행합니다...")

    # 랜덤 데이터셋을 생성하는 내부 함수 정의
    def create_random_dataset(num_samples=1000):
        # 정규분포 랜덤 노이즈로 이미지 형태 데이터 생성
        random_data = torch.randn(num_samples, 3, image_size, image_size)
        # 값을 적절한 범위로 조정하고 [0, 1] 범위로 클램핑
        random_data = torch.clamp((random_data * 0.2) + 0.5, 0, 1)
        # (이미지, 라벨) 튜플 형태의 리스트 생성 (라벨은 임의로 0)
        random_dataset = [(img, 0) for img in random_data]
        # 생성된 랜덤 데이터셋 반환
        return random_dataset

    # PyTorch Dataset 클래스를 상속한 커스텀 랜덤 데이터셋 클래스
    class RandomDataset(torch.utils.data.Dataset):
        # 생성자: 데이터를 받아서 저장
        def __init__(self, data):
            self.data = data

        # 데이터셋 크기 반환 메서드
        def __len__(self):
            return len(self.data)

        # 인덱스에 해당하는 데이터 항목 반환 메서드
        def __getitem__(self, idx):
            return self.data[idx]

    # 랜덤 데이터셋 객체 생성
    random_dataset = RandomDataset(create_random_dataset())
    # 랜덤 데이터셋용 DataLoader 생성
    dataloader = DataLoader(random_dataset, batch_size=batch_size, shuffle=True)
    # 랜덤 데이터셋 생성 완료 메시지 출력
    print(f"임의 데이터셋 생성 완료: {len(random_dataset)} 이미지")

# ==================== 가중치 초기화 함수 섹션 ====================
# DCGAN 논문에서 권장하는 가중치 초기화 함수
def weights_init(m):
    # 모듈의 클래스 이름을 문자열로 가져옴
    classname = m.__class__.__name__
    # 클래스 이름에 'Conv'가 포함되어 있으면 (Convolution 레이어)
    if classname.find('Conv') != -1:
        # 가중치를 평균 0.0, 표준편차 0.02인 정규분포로 초기화
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    # 클래스 이름에 'BatchNorm'이 포함되어 있으면 (Batch Normalization 레이어)
    elif classname.find('BatchNorm') != -1:
        # 가중치를 평균 1.0, 표준편차 0.02인 정규분포로 초기화
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        # bias를 0으로 초기화
        nn.init.constant_(m.bias.data, 0)

# ==================== Generator 모델 정의 섹션 ====================
# 노이즈 벡터를 이미지로 변환하는 생성자 네트워크 클래스
class Generator(nn.Module):
    # 생성자 메서드
    def __init__(self):
        # 부모 클래스 초기화
        super(Generator, self).__init__()
        # Sequential 컨테이너로 레이어들을 순차적으로 연결
        self.main = nn.Sequential(
            # 첫 번째 전치 합성곱 레이어: nz(100) -> ngf*8(512) 채널
            # 입력: (배치크기, 100, 1, 1) -> 출력: (배치크기, 512, 4, 4)
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            # 배치 정규화로 학습 안정성 향상
            nn.BatchNorm2d(ngf * 8),
            # ReLU 활성화 함수 (inplace=True: 메모리 효율성)
            nn.ReLU(True),

            # 두 번째 전치 합성곱 레이어: ngf*8(512) -> ngf*4(256) 채널
            # 입력: (배치크기, 512, 4, 4) -> 출력: (배치크기, 256, 8, 8)
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            # 배치 정규화
            nn.BatchNorm2d(ngf * 4),
            # ReLU 활성화 함수
            nn.ReLU(True),

            # 세 번째 전치 합성곱 레이어: ngf*4(256) -> ngf*2(128) 채널
            # 입력: (배치크기, 256, 8, 8) -> 출력: (배치크기, 128, 16, 16)
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            # 배치 정규화
            nn.BatchNorm2d(ngf * 2),
            # ReLU 활성화 함수
            nn.ReLU(True),

            # 네 번째 전치 합성곱 레이어: ngf*2(128) -> ngf(64) 채널
            # 입력: (배치크기, 128, 16, 16) -> 출력: (배치크기, 64, 32, 32)
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            # 배치 정규화
            nn.BatchNorm2d(ngf),
            # ReLU 활성화 함수
            nn.ReLU(True),

            # 마지막 전치 합성곱 레이어: ngf(64) -> 3(RGB) 채널
            # 입력: (배치크기, 64, 32, 32) -> 출력: (배치크기, 3, 64, 64)
            nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
            # Tanh 활성화 함수로 출력을 [-1, 1] 범위로 제한
            nn.Tanh()
        )

    # 순전파 메서드: 입력을 네트워크에 통과시켜 출력 생성
    def forward(self, input):
        # Sequential 네트워크에 입력을 통과시킨 결과 반환
        return self.main(input)

# ==================== Discriminator 모델 정의 섹션 ====================
# 이미지가 진짜인지 가짜인지 판별하는 판별자 네트워크 클래스
class Discriminator(nn.Module):
    # 생성자 메서드
    def __init__(self):
        # 부모 클래스 초기화
        super(Discriminator, self).__init__()
        # Sequential 컨테이너로 레이어들을 순차적으로 연결
        self.main = nn.Sequential(
            # 첫 번째 합성곱 레이어: 3(RGB) -> ndf(64) 채널
            # 입력: (배치크기, 3, 64, 64) -> 출력: (배치크기, 64, 32, 32)
            nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
            # LeakyReLU 활성화 함수 (음수 기울기 0.2, inplace=True)
            nn.LeakyReLU(0.2, inplace=True),

            # 두 번째 합성곱 레이어: ndf(64) -> ndf*2(128) 채널
            # 입력: (배치크기, 64, 32, 32) -> 출력: (배치크기, 128, 16, 16)
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            # 배치 정규화
            nn.BatchNorm2d(ndf * 2),
            # LeakyReLU 활성화 함수
            nn.LeakyReLU(0.2, inplace=True),

            # 세 번째 합성곱 레이어: ndf*2(128) -> ndf*4(256) 채널
            # 입력: (배치크기, 128, 16, 16) -> 출력: (배치크기, 256, 8, 8)
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            # 배치 정규화
            nn.BatchNorm2d(ndf * 4),
            # LeakyReLU 활성화 함수
            nn.LeakyReLU(0.2, inplace=True),

            # 네 번째 합성곱 레이어: ndf*4(256) -> ndf*8(512) 채널
            # 입력: (배치크기, 256, 8, 8) -> 출력: (배치크기, 512, 4, 4)
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            # 배치 정규화
            nn.BatchNorm2d(ndf * 8),
            # LeakyReLU 활성화 함수
            nn.LeakyReLU(0.2, inplace=True),

            # 마지막 합성곱 레이어: ndf*8(512) -> 1 채널
            # 입력: (배치크기, 512, 4, 4) -> 출력: (배치크기, 1, 1, 1)
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            # Sigmoid 활성화 함수로 출력을 [0, 1] 확률값으로 변환
            nn.Sigmoid()
        )

    # 순전파 메서드: 이미지를 입력받아 진짜/가짜 확률 반환
    def forward(self, input):
        # 네트워크 출력을 1차원으로 변환하여 확률값 반환
        # view(-1, 1): (배치크기, 1, 1, 1) -> (배치크기, 1)
        # squeeze(1): (배치크기, 1) -> (배치크기,)
        return self.main(input).view(-1, 1).squeeze(1)

# ==================== 모델 객체 생성 및 초기화 섹션 ====================
# 생성자 네트워크 객체 생성 및 지정된 장치로 이동
netG = Generator().to(device)
# 판별자 네트워크 객체 생성 및 지정된 장치로 이동
netD = Discriminator().to(device)

# 생성자 네트워크의 모든 레이어에 가중치 초기화 함수 적용
netG.apply(weights_init)
# 판별자 네트워크의 모든 레이어에 가중치 초기화 함수 적용
netD.apply(weights_init)

# 생성자 모델 구조 출력
print("생성자 모델 구조:")
print(netG)
# 판별자 모델 구조 출력
print("\n판별자 모델 구조:")
print(netD)

# ==================== 손실 함수 및 옵티마이저 설정 섹션 ====================
# 이진 교차 엔트로피 손실 함수 (GAN의 표준 손실 함수)
criterion = nn.BCELoss()

# 학습 중 생성 품질 확인용 고정 노이즈 벡터 생성
# (64개 샘플, nz 차원, 1x1 크기, 지정된 장치에 배치)
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# 진짜 이미지에 대한 라벨 값 (1: 진짜)
real_label = 1
# 가짜 이미지에 대한 라벨 값 (0: 가짜)
fake_label = 0

# 판별자용 Adam 옵티마이저 생성 (학습률 lr, 베타 파라미터 설정)
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
# 생성자용 Adam 옵티마이저 생성
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# ==================== 이미지 시각화 함수 섹션 ====================
# 이미지 텐서를 시각화하는 함수
def visualize_images(images, title=None, display_in_notebook=True):
    # 이미지 픽셀값을 [-1, 1] 범위에서 [0, 1] 범위로 변환
    images = (images + 1) / 2.0

    # 여러 이미지를 격자 형태로 배열 (패딩 2픽셀, 정규화 없음)
    grid = torchvision.utils.make_grid(images, padding=2, normalize=False)
    # PyTorch 텐서를 numpy 배열로 변환하고 채널 순서 변경
    # (채널, 높이, 너비) -> (높이, 너비, 채널)
    img = grid.permute(1, 2, 0).cpu().numpy()

    # matplotlib 그림 생성 (8x8 인치 크기)
    plt.figure(figsize=(8, 8))
    # 이미지 배열을 화면에 표시
    plt.imshow(img)
    # 제목이 제공된 경우 설정
    if title:
        plt.title(title)
    # 축 눈금과 라벨 숨기기
    plt.axis('off')

    # 노트북 환경에서 이미지 표시 여부 결정
    if display_in_notebook:
        plt.show()

    # 처리된 이미지 배열 반환
    return img

# ==================== 결과 저장 폴더 생성 섹션 ====================
# 학습 결과 이미지를 저장할 'results' 폴더 생성
os.makedirs("results", exist_ok=True)
# 모델 체크포인트를 저장할 'checkpoints' 폴더 생성
os.makedirs("checkpoints", exist_ok=True)

# ==================== 학습 기록 리스트 초기화 섹션 ====================
# 생성자 손실값들을 저장할 리스트
G_losses = []
# 판별자 손실값들을 저장할 리스트
D_losses = []
# 각 에폭별 생성된 이미지들을 저장할 리스트
img_list = []

# 학습 시작 알림 메시지
print("학습 시작...")

# ==================== 프로그레스 바 함수 섹션 ====================
# 학습 진행률을 시각적으로 보여주는 프로그레스 바 함수
def progress_bar(current, total, bar_length=50):
    # 전체 작업 중 현재 진행된 비율 계산
    fraction = current / total
    # 진행된 부분을 '=' 문자로 표현
    arrow = int(fraction * bar_length) * '='
    # 남은 부분을 공백으로 표현
    padding = (bar_length - len(arrow)) * ' '
    # 프로그레스 바 문자열 생성 및 반환
    return f"[{arrow}{padding}] {int(fraction * 100)}%"

# ==================== GAN 학습 메인 루프 섹션 ====================
# 에폭 수만큼 반복하는 메인 학습 루프
for epoch in range(num_epochs):
    # 현재 에폭의 시작 시간 기록
    start_time = time.time()

    # 데이터로더에서 배치 단위로 데이터를 순회
    for i, data in enumerate(dataloader, 0):

        # ==================== 판별자 학습 파트 ====================
        # 판별자의 그래디언트를 0으로 초기화
        netD.zero_grad()

        # 데이터가 리스트 형태인지 확인하고 실제 이미지 추출
        if isinstance(data, list) and len(data) == 2:
            # 첫 번째 요소가 이미지라고 가정하고 장치로 이동
            real_cpu = data[0].to(device)
        else:
            # 데이터가 직접 이미지 텐서인 경우
            real_cpu = data[0].to(device)

        # 현재 배치의 크기 확인
        batch_size = real_cpu.size(0)
        # 진짜 이미지에 대한 라벨 텐서 생성 (모든 값이 real_label=1)
        label = torch.full((batch_size,), real_label, dtype=torch.float, device=device)

        # 진짜 이미지를 판별자에 통과시켜 예측값 얻기
        output = netD(real_cpu)
        # 진짜 이미지에 대한 판별자 손실 계산
        errD_real = criterion(output, label)
        # 역전파로 그래디언트 계산
        errD_real.backward()
        # 판별자가 진짜 이미지를 진짜로 예측한 평균 확률 저장
        D_x = output.mean().item()

        # 생성자 입력용 랜덤 노이즈 벡터 생성
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        # 생성자로 가짜 이미지 생성
        fake = netG(noise)
        # 가짜 이미지에 대한 라벨을 fake_label=0으로 설정
        label.fill_(fake_label)

        # 가짜 이미지를 판별자에 통과 (.detach()로 생성자 그래디언트 차단)
        output = netD(fake.detach())
        # 가짜 이미지에 대한 판별자 손실 계산
        errD_fake = criterion(output, label)
        # 역전파로 그래디언트 계산
        errD_fake.backward()
        # 판별자가 가짜 이미지를 진짜로 잘못 예측한 평균 확률 저장
        D_G_z1 = output.mean().item()

        # 판별자의 총 손실 = 진짜 이미지 손실 + 가짜 이미지 손실
        errD = errD_real + errD_fake
        # 판별자 가중치 업데이트
        optimizerD.step()

        # ==================== 생성자 학습 파트 ====================
        # 생성자의 그래디언트를 0으로 초기화
        netG.zero_grad()
        # 생성자 학습용: 가짜 이미지의 라벨을 real_label=1로 설정
        # (생성자는 판별자가 가짜를 진짜로 착각하게 만들고 싶어함)
        label.fill_(real_label)

        # 생성자가 만든 가짜 이미지를 다시 판별자에 통과
        output = netD(fake)
        # 생성자 손실 계산 (판별자가 가짜를 진짜로 판단하길 원함)
        errG = criterion(output, label)
        # 역전파로 생성자 그래디언트 계산
        errG.backward()
        # 생성자 학습 후 판별자가 가짜를 진짜로 예측한 평균 확률
        D_G_z2 = output.mean().item()

        # 생성자 가중치 업데이트
        optimizerG.step()

        # ==================== 손실 기록 및 진행상황 출력 섹션 ====================
        # 현재 배치의 손실값들을 리스트에 추가
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # 10 배치마다 진행 상황 출력
        if i % 10 == 0:
            # 프로그레스 바 문자열 생성
            prog = progress_bar(i, len(dataloader))
            # 현재 에폭, 배치, 손실값들, 판별 확률들을 한 줄에 출력
            print(f'\r에폭 [{epoch+1}/{num_epochs}] 배치 {prog} '
                  f'Loss_D: {errD.item():.4f} Loss_G: {errG.item():.4f} '
                  f'D(x): {D_x:.4f} D(G(z)): {D_G_z1:.4f}/{D_G_z2:.4f}', end='')

    # 에폭 완료 후 줄바꿈
    print(' ')

    # ==================== 에폭별 이미지 생성 및 저장 섹션 ====================
    # 그래디언트 계산을 비활성화하여 메모리 절약
    with torch.no_grad():
        # 고정된 노이즈로 이미지 생성하고 CPU로 이동
        fake = netG(fixed_noise).detach().cpu()

    # 생성된 이미지를 리스트에 추가 (학습 진행 과정 추적용)
    img_list.append(fake)

    # 생성된 이미지를 시각화
    img = visualize_images(fake, title=f'epoch {epoch+1} Result')

    # 결과 이미지를 PNG 파일로 저장
    plt.savefig(f'results/fake_dogs_epoch_{epoch+1}.png')
    # 현재 플롯을 닫아 메모리 해제
    plt.close()

    # ==================== 모델 체크포인트 저장 섹션 ====================
    # 5 에폭마다 또는 마지막 에폭에서 모델 저장
    if (epoch + 1) % 5 == 0 or (epoch + 1) == num_epochs:
        # 모델 상태, 옵티마이저 상태, 손실 기록 등을 딕셔너리로 저장
        torch.save({
            'generator': netG.state_dict(),        # 생성자 가중치
            'discriminator': netD.state_dict(),    # 판별자 가중치
            'optimizerG': optimizerG.state_dict(), # 생성자 옵티마이저 상태
            'optimizerD': optimizerD.state_dict(), # 판별자 옵티마이저 상태
            'epoch': epoch,                        # 현재 에폭 번호
            'G_losses': G_losses,                  # 생성자 손실 기록
            'D_losses': D_losses,                  # 판별자 손실 기록
        }, f'checkpoints/gan_model_epoch_{epoch+1}.pth')
        # 체크포인트 저장 완료 메시지
        print(f"모델 체크포인트 저장: 에폭 {epoch+1}")

    # ==================== 에폭 소요시간 계산 및 출력 섹션 ====================
    # 현재 에폭의 소요 시간 계산
    elapsed = time.time() - start_time
    # 에폭 완료 및 소요 시간 출력
    print(f'에폭 {epoch+1} 완료, 소요 시간: {elapsed:.2f}초')

# 전체 학습 완료 메시지
print("학습 완료!")

# ==================== 손실 그래프 시각화 섹션 ====================
# 손실 그래프를 그리기 위한 figure 생성
plt.figure(figsize=(10, 5))
# 그래프 제목 설정
plt.title("Loss of Generator and Discriminator")
# 생성자 손실을 선 그래프로 플롯
plt.plot(G_losses, label="Generator")
# 판별자 손실을 선 그래프로 플롯
plt.plot(D_losses, label="Discriminator")
# x축 라벨 설정
plt.xlabel("Repeat")
# y축 라벨 설정
plt.ylabel("Loss")
# 범례 표시
plt.legend()
# 그래프를 PNG 파일로 저장
plt.savefig('results/loss_plot.png')
# 그래프를 화면에 표시
plt.show()
# 현재 플롯 닫기
plt.close()

# ==================== 학습 진행 과정 시각화 섹션 ====================
# 각 에폭별 생성 이미지들을 그리드로 시각화
plt.figure(figsize=(12, 12))
# 그리드의 행 수 계산 (제곱근 이용)
rows = int(np.sqrt(num_epochs))
# 그리드의 열 수 계산 (올림 이용)
cols = int(np.ceil(num_epochs / rows))

# 에폭 수 또는 저장된 이미지 수 중 작은 값만큼 반복
for i in range(min(num_epochs, len(img_list))):
    # 서브플롯 위치 설정 (i+1번째 위치)
    plt.subplot(rows, cols, i + 1)
    # 축 숨기기
    plt.axis('off')
    # 서브플롯 제목 설정 (에폭 번호)
    plt.title(f'epoch {i+1}')

    # 저장된 이미지가 있는 경우에만 처리
    if i < len(img_list):
        # 해당 에폭의 이미지 중 처음 16개만 격자로 배열
        img = torchvision.utils.make_grid(img_list[i][:16], padding=2, normalize=True)
        # 텐서를 numpy 배열로 변환하고 채널 순서 변경하여 표시
        plt.imshow(np.transpose(img.cpu().numpy(), (1, 2, 0)))

# 서브플롯 간 간격 자동 조정
plt.tight_layout()
# 전체 진행 과정 이미지를 파일로 저장
plt.savefig('results/progress.png')
# 화면에 표시
plt.show()

# ==================== 새 이미지 생성 함수 섹션 ====================
# 학습된 생성자로 새로운 이미지를 생성하는 함수
def generate_new_dogs(num_images=16):
    # 생성자를 평가 모드로 설정 (Dropout, BatchNorm 등 비활성화)
    netG.eval()

    # 그래디언트 계산 비활성화
    with torch.no_grad():
        # 새로운 랜덤 노이즈 벡터 생성
        noise = torch.randn(num_images, nz, 1, 1, device=device)

        # 생성자로 새 가짜 이미지 생성 후 CPU로 이동
        fake_dogs = netG(noise).detach().cpu()

        # 생성된 이미지들을 시각화
        visualize_images(fake_dogs, title=f'Generated Dog Image {num_images}')
        # 최종 생성 이미지를 파일로 저장
        plt.savefig('results/final_generated_dogs.png')

        # 생성 완료 메시지 출력
        print(f"{num_images}개의 새로운 강아지 이미지를 생성했습니다. ('results/final_generated_dogs.png'에 저장)")

        # 생성된 이미지 데이터 반환
        return fake_dogs

# ==================== 최종 이미지 생성 섹션 ====================
# 최종 모델 이미지 생성 안내 메시지
print("\n최종 모델로 새 이미지 생성:")
# 16개의 새로운 강아지 이미지 생성
generate_new_dogs(16)

# ==================== 결과 다운로드 안내 함수 섹션 ====================
# 생성된 결과 파일들을 다운로드하는 방법을 안내하는 함수
def download_results():
    # 다운로드 방법 안내 메시지들 출력
    print("\n생성된 결과 다운로드:")
    print("1. 왼쪽 사이드바에서 파일 탭(📁)을 클릭합니다.")
    print("2. 'results' 폴더에서 생성된 이미지를 다운로드합니다.")
    print("또는 아래 코드를 실행하여 결과 파일을 다운로드할 수 있습니다:")
    print("files.download('results/final_generated_dogs.png')")
    print("files.download('results/progress.png')")
    print("files.download('results/loss_plot.png')")

# ==================== 실습 완료 섹션 ====================
# 실습 완료 메시지 출력
print("\n=== GAN 실습2 완료 ===")
print("생성된 모든 결과는 'results' 폴더에 저장되었습니다.")
# 결과 다운로드 안내 함수 호출
download_results()